In [ ]:
# CISB5123 - TEXT ANALYTICS (SECTION 03)
# AHMAD AMIRUL AIZAD BIN ROSMADI [IS01082507]
# MUHAMMMAD NABIL BIN MUHAMMAD [IS01082117]

In [1]:
pip install selenium webdriver-manager

Note: you may need to restart the kernel to use updated packages.


In [48]:
import csv
import time
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from webdriver_manager.chrome import ChromeDriverManager

# Lazada Product URL
PRODUCT_URL = "https://www.lazada.com.my/products/thermos-cup-set-double-layer-stainless-steel-vacuum-thermos-bottle-travel-mug-water-bottle-cup-set-gift-set-i3747418031-s21229207831.html"

# Chrome options
options = Options()
options.add_argument("--headless")  # Run headless to avoid UI display
options.add_argument("--disable-gpu")
options.add_argument("--no-sandbox")
options.add_argument("--disable-notifications")
options.add_argument("--disable-popup-blocking")

# Initialize driver
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)
wait = WebDriverWait(driver, 10)

def close_popups():
    """Close any popups or overlays that block navigation."""
    try:
        close_btns = driver.find_elements(By.CSS_SELECTOR, "button[class*='close'], div[class*='overlay']")
        for btn in close_btns:
            driver.execute_script("arguments[0].click();", btn)
            time.sleep(2)
    except:
        pass


def get_reviews():
    """Scrape reviews from Lazada product page."""
    driver.get(PRODUCT_URL)
    time.sleep(5)  # Allow page to load

    reviews_data = []
    page = 1
    max_pages = 5  # Control the number of pages to scrape

    while page <= max_pages:
        print(f"Scraping page {page}...")

        # Close any popups
        close_popups()

        # Scroll to load more reviews
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(5)

        # Extract reviews
        reviews = driver.find_elements(By.CSS_SELECTOR, "div.item")

        for review in reviews:
            try:
                name = review.find_element(By.CSS_SELECTOR, "div.middle span").text or "Anonymous"
                date = review.find_element(By.CSS_SELECTOR, "div.top span").text or "Unknown"
                content = review.find_element(By.CSS_SELECTOR, "div.item-content div.content").text or "No content"
                reviews_data.append([name, date, content])
            except:
                continue

        # Handle 'Next' button properly
        next_buttons = driver.find_elements(By.CSS_SELECTOR, "button.next-btn.next-btn-normal.next-btn-medium.next-pagination-item.next")
        if next_buttons:
            driver.execute_script("arguments[0].click();", next_buttons[0])
            time.sleep(5)
        else:
            print("No more pages.")
            break

        page += 1

    return reviews_data


def save_to_csv(data, filename="lazada_reviews.csv"):
    """Save scraped reviews to CSV file."""
    with open(filename, "w", newline="", encoding="utf-8") as file:
        writer = csv.writer(file)
        writer.writerow(["Reviewer Name", "Review Date", "Review Content"])
        writer.writerows(data)


if __name__ == "__main__":
    reviews = get_reviews()
    save_to_csv(reviews)
    print(f"Scraped {len(reviews)} reviews and saved to lazada_reviews.csv.")
    driver.quit()


Scraping page 1...
Scraping page 2...
Scraping page 3...
Scraping page 4...
Scraping page 5...
Scraped 25 reviews and saved to lazada_reviews.csv.
